Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Loading datasets

In [ ]:
cats=pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops=pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
items=pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test=pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

Count of each datasets

In [ ]:
print('There are ' + str(len(items)) + ' different items')
print('There are ' + str(len(cats)) + ' different item categories')
print('There are ' + str(len(shops)) + ' different shops')

Display of each dataset

In [ ]:
items.head()

In [ ]:
cats.head()

In [ ]:
shops.head()

In [ ]:
test.head()

In [ ]:
train.head()

Checking Missing values

In [ ]:
train.isnull().values.any()

In [ ]:
items.isnull().values.any()

In [ ]:
cats.isnull().values.any()

In [ ]:
shops.isnull().values.any()

In [ ]:
test.isnull().values.any()

Checking for duplicate values

In [ ]:
train.duplicated().any()

In [ ]:
items.duplicated().any()

In [ ]:
cats.duplicated().any()

In [ ]:
shops.duplicated().any()

In [ ]:
test.duplicated().any()

Train data cleaning

In [ ]:
print("Shape of the data: ", train.shape)
print("\n")
display(train.head())
print("No of Unique date block num: ", train['date_block_num'].nunique())
print("No of Unique shop id: ", train['shop_id'].nunique())
print("No of Unique item id: ", train['item_id'].nunique())
print("Any Null Values?")
print(train.isnull().sum())

print('\nBasic Stats of Item Price')
display(train['item_price'].describe())

print('\nBasic Stats of Item_count_day')
display(train['item_cnt_day'].describe())

print("\nThere are few negative values too in the sale and count \
does they represent any return? How many such values are there?")

print('\nLets check the negative values')

display(train[train['item_price']<0])
display(train[train['item_cnt_day']<0])

print("\nStats of negative item count day")
display(train.loc[train['item_cnt_day']<0, 'item_cnt_day'].describe())

In [ ]:
# Filling the negative value with median value
train[train['item_price']< 0] = train['item_price'].median()  #replaced all negative values with median values
train[train['item_cnt_day']< 0] = 0

In [ ]:
#splitting the date column
import datetime as dt
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y' , infer_datetime_format=True)

train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year

train.drop('date', axis=1, inplace=True)

train.head()

In [ ]:
#dropping the date block num and item price
train.drop(['date_block_num', 'item_price'], axis=1, inplace=True)
train.head()

In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train, test_size=0.2, random_state=29)

In [ ]:
train_set.head()

In [ ]:
test_set.head()

In [ ]:
train.shape

In [ ]:
train_set.shape

In [ ]:
test_set.shape

In [ ]:
#defining dependent variable and independent variable in train set
x = train_set.drop('item_cnt_day', axis=1)
y = train_set['item_cnt_day'].copy()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
#scaling the train dataset
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_prepared = min_max_scaler.fit_transform(x)

In [ ]:
#Random forest regressor model building
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

In [ ]:
X_test = test_set.drop('item_cnt_day', axis=1)
y_test = test_set['item_cnt_day'].copy()

In [ ]:
X_test_prepared = min_max_scaler.transform(X_test)

In [ ]:
y_predicted = RF_model.predict(X_test_prepared)

print("Predictions:")
for i in y_predicted[:10]:
    print(i)
    
print("Orginal test:")
y_test.iloc[:10]

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)

print("MAE=", mae)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predicted)

print("RMSE=", np.sqrt(mse))

In [ ]:
df_test = test.copy()

In [ ]:
df_test['day'] = 1
df_test['month'] = 11
df_test['year'] = 2015

In [ ]:
df_test.drop('ID', axis=1, inplace=True)
df_test

In [ ]:
df_test_prepared = min_max_scaler.transform(df_test)

In [ ]:
l = list()

for i in range(1, 31):
  df_test['day'] = i
  df_test_prepared = min_max_scaler.transform(df_test)
  l.append(RF_model.predict(df_test_prepared))

In [ ]:
sample_subm=pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
RF_subm = sample_subm.copy()
RF_subm['item_cnt_month'] = sum(l)

In [ ]:
RF_subm.to_csv("submission.csv", index=False)